# Carga de Datos

## Objetivo
Cargar la base de datos de créditos desde el archivo fuente, realizar validaciones iniciales y preparar los datos para el análisis exploratorio.

## Dataset
- **Archivo:** Base_de_datos.xlsx (o train.csv)
- **Origen:** Sistema de créditos interno
- **Período:** Diciembre 2024 - Enero 2026
- **Registros esperados:** ~10,000-11,000
- **Variables:** 23 columnas

## 1. Configuración Inicial

In [ ]:
# Importar libre        rías necesarias
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Librerías importadas correctamente")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✓ Librerías importadas correctamente
Pandas version: 2.3.3
NumPy version: 2.4.2


## 2. Definir Rutas del Proyecto

**IMPORTANTE:** Como el notebook está en `src/`, la raíz del proyecto es el directorio actual (no necesitamos `.parent`)

In [10]:
# Definir rutas relativas desde src/ (donde está este notebook)
# Como estamos EN la raíz del proyecto (mlops_pipeline/src/), 
# usamos Path.cwd() directamente

PROJECT_ROOT = Path.cwd()  # Ya estamos en mlops_pipeline/

# Crear carpetas de datos si no existen
DATA_DIR = PROJECT_ROOT / 'data'
DATA_RAW = DATA_DIR / 'raw'
DATA_INTERIM = DATA_DIR / 'interim'
DATA_PROCESSED = DATA_DIR / 'processed'

# Crear directorios
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("Rutas del proyecto:")
print(f"  ROOT:           {PROJECT_ROOT}")
print(f"  DATA_RAW:       {DATA_RAW}")
print(f"  DATA_INTERIM:   {DATA_INTERIM}")
print(f"  DATA_PROCESSED: {DATA_PROCESSED}")
print(f"\nCarpetas creadas exitosamente ✓")

Rutas del proyecto:
  ROOT:           d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src
  DATA_RAW:       d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\raw
  DATA_INTERIM:   d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\interim
  DATA_PROCESSED: d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\processed

Carpetas creadas exitosamente ✓


## 3. Identificar Archivo de Datos

Buscamos el archivo de datos. Puede ser:
- `Base_de_datos.xlsx` (en data/raw/)
- `train.csv` (en la raíz o en data/raw/)

In [11]:
# Posibles ubicaciones del archivo
possible_files = [
    DATA_RAW / 'Base_de_datos.xlsx',
    DATA_RAW / 'train.csv',
    PROJECT_ROOT / 'train.csv',  # Por si está en la raíz
    PROJECT_ROOT / 'Base_de_datos.xlsx',
]

# Buscar el primer archivo que exista
FILE_PATH = None
for file in possible_files:
    if file.exists():
        FILE_PATH = file
        print(f"✓ Archivo encontrado: {file.name}")
        print(f"  Ubicación: {file}")
        break

if FILE_PATH is None:
    print("⚠️ No se encontró el archivo de datos.")
    print("\nPor favor, coloca uno de estos archivos:")
    print(f"  - Base_de_datos.xlsx en: {DATA_RAW}")
    print(f"  - train.csv en: {DATA_RAW}")
    raise FileNotFoundError("Archivo de datos no encontrado")
else:
    print(f"\n¿Archivo existe? {FILE_PATH.exists()} ✓")

✓ Archivo encontrado: Base_de_datos.xlsx
  Ubicación: d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\raw\Base_de_datos.xlsx

¿Archivo existe? True ✓


## 4. Función de Carga de Datos

In [12]:
def load_data(file_path):
    """
    Carga datos desde archivo CSV o Excel.
    
    Args:
        file_path (Path): Ruta al archivo de datos
        
    Returns:
        pd.DataFrame: DataFrame con los datos cargados
    """
    print(f"\nCargando datos desde: {file_path.name}")
    print(f"Ruta completa: {file_path}")
    
    if file_path.suffix == '.csv':
        df = pd.read_csv(file_path)
        print("✓ Archivo CSV cargado exitosamente")
    elif file_path.suffix in ['.xlsx', '.xls']:
        df = pd.read_excel(file_path)
        print("✓ Archivo Excel cargado exitosamente")
    else:
        raise ValueError(f"Formato no soportado: {file_path.suffix}")
    
    print(f"Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    
    return df

## 5. Cargar Dataset

In [13]:
# Cargar datos
df = load_data(FILE_PATH)


Cargando datos desde: Base_de_datos.xlsx
Ruta completa: d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\raw\Base_de_datos.xlsx
✓ Archivo Excel cargado exitosamente
Dimensiones: 10,763 filas × 23 columnas


## 6. Validación Inicial del Dataset

### 6.1 Verificar Estructura Esperada

In [ ]:
# Columnas esperadas
EXPECTED_COLUMNS = [
    'tipo_credito', 'fecha_prestamo', 'capital_prestado', 'plazo_meses',
    'edad_cliente', 'tipo_laboral', 'salario_cliente', 'total_otros_prestamos',
    'cuota_pactada', 'puntaje', 'puntaje_datacredito', 'cant_creditosvigentes',
    'huella_consulta', 'saldo_mora', 'saldo_total', 'saldo_principal',
    'saldo_mora_codeudor', 'creditos_sectorFinanciero', 'creditos_sectorCooperativo',
    'creditos_sectorReal', 'promedio_ingresos_datacredito', 'tendencia_ingresos',
    'Pago_atiempo'
]

print("=" * 80)
print("VALIDACIÓN DE ESTRUCTURA")
print("=" * 80)

# Verificar número de columnas
print(f"\n1. Número de columnas:")
print(f"   Esperadas: {len(EXPECTED_COLUMNS)}")
print(f"   Encontradas: {len(df.columns)}")
if len(df.columns) == len(EXPECTED_COLUMNS):
    print("   ✓ CORRECTO")
else:
    print(f"   * DIFERENCIA DETECTADA: {abs(len(df.columns) - len(EXPECTED_COLUMNS))} columnas")

# Verificar nombres de columnas
print(f"\n2. Nombres de columnas:")
missing_cols = set(EXPECTED_COLUMNS) - set(df.columns)
extra_cols = set(df.columns) - set(EXPECTED_COLUMNS)

if missing_cols:
    print(f"   * Columnas faltantes: {missing_cols}")
if extra_cols:
    print(f"   * Columnas extra: {extra_cols}")
if not missing_cols and not extra_cols:
    print("   ✓ Todas las columnas esperadas están presentes")

# Verificar número de registros
print(f"\n3. Número de registros:")
print(f"   Registros: {len(df):,}")
if 10000 <= len(df) <= 12000:
    print("   ✓ Dentro del rango esperado (10,000-12,000)")
elif len(df) < 10000:
    print(f"   * Menos registros de lo esperado ({len(df):,} < 10,000)")
else:
    print(f"   ¡ Más registros de lo esperado ({len(df):,} > 12,000)")

print("\n" + "=" * 80)

VALIDACIÓN DE ESTRUCTURA

1. Número de columnas:
   Esperadas: 23
   Encontradas: 23
   ✓ CORRECTO

2. Nombres de columnas:
   ✓ Todas las columnas esperadas están presentes

3. Número de registros:
   Registros: 10,763
   ✓ Dentro del rango esperado (10,000-12,000)



### 6.2 Vista Previa de los Datos

In [15]:
# Primeras filas
print("\nPrimeras 10 filas del dataset:")
display(df.head(10))


Primeras 10 filas del dataset:


,tipo_credito,fecha_prestamo,capital_prestado,plazo_meses,edad_cliente,tipo_laboral,salario_cliente,total_otros_prestamos,cuota_pactada,puntaje,puntaje_datacredito,cant_creditosvigentes,huella_consulta,saldo_mora,saldo_total,saldo_principal,saldo_mora_codeudor,creditos_sectorFinanciero,creditos_sectorCooperativo,creditos_sectorReal,promedio_ingresos_datacredito,tendencia_ingresos,Pago_atiempo
0,7,2024-12-21 11:31:35,3692160.0,10,42,Independiente,8000000,2500000,341296,88.768094,695.0,10,5,0.0,51258.0,51258.0,0.0,5,0,0,908526.0,Estable,1
1,4,2025-04-22 09:47:35,840000.0,6,60,Empleado,3000000,2000000,124876,95.227787,789.0,3,1,0.0,8673.0,8673.0,0.0,0,0,2,939017.0,Creciente,1
2,9,2026-01-08 12:22:40,5974028.4,10,36,Independiente,4036000,829000,529554,47.613894,740.0,4,5,0.0,18702.0,18702.0,0.0,3,0,0,NaN,NaN,0
3,4,2025-08-04 12:04:10,1671240.0,6,48,Empleado,1524547,498000,252420,95.227787,837.0,4,4,0.0,15782.0,15782.0,0.0,3,0,0,1536193.0,Creciente,1
4,9,2025-04-26 11:24:26,2781636.0,11,44,Empleado,5000000,4000000,217037,95.227787,771.0,4,6,0.0,204804.0,204804.0,0.0,3,0,1,933473.0,Creciente,1
5,4,2025-06-10 08:54:01,1031928.0,12,32,Empleado,2800000,800000,82872,95.227787,721.0,10,13,0.0,24399.0,24399.0,0.0,2,0,8,2808474.0,Creciente,1
6,4,2025-08-09 13:00:44,3064280.4,6,68,Independiente,1000000,22005000,461231,95.227787,814.0,11,8,0.0,211775.0,211775.0,0.0,6,3,0,969508.0,Creciente,1
7,4,2025-08-18 12:49:19,3619560.0,6,31,Empleado,3782303,305000,542079,95.227787,760.0,1,5,0.0,12078.0,12078.0,0.0,0,1,0,3782303.0,Creciente,1
8,9,2025-05-30 09:11:18,2134827.6,10,31,Empleado,14500000,8000000,181732,95.227787,737.0,0,2,0.0,0.0,NaN,NaN,0,0,0,14007850.0,Creciente,1
9,4,2024-12-31 14:32:59,8400000.0,6,45,Empleado,14000000,5000000,1166667,95.227787,752.0,4,3,0.0,125383.0,125383.0,0.0,2,0,0,28889623.0,Creciente,1


In [16]:
# Últimas filas
print("\nÚltimas 5 filas del dataset:")
display(df.tail())


Últimas 5 filas del dataset:


,tipo_credito,fecha_prestamo,capital_prestado,plazo_meses,edad_cliente,tipo_laboral,salario_cliente,total_otros_prestamos,cuota_pactada,puntaje,puntaje_datacredito,cant_creditosvigentes,huella_consulta,saldo_mora,saldo_total,saldo_principal,saldo_mora_codeudor,creditos_sectorFinanciero,creditos_sectorCooperativo,creditos_sectorReal,promedio_ingresos_datacredito,tendencia_ingresos,Pago_atiempo
10758,9,2025-01-19 16:18:28,2414886.0,10,29,Independiente,3000000,300000,204819,13.134355,747.0,2,2,172.0,1112.0,1112.0,NaN,0,0,1,NaN,NaN,0
10759,4,2025-01-10 16:40:21,2916000.0,24,27,Empleado,2500000,400000,127460,55.973342,714.0,5,4,0.0,9771.0,9771.0,0.0,1,0,4,1958333.0,Creciente,0
10760,4,2025-06-19 14:28:47,4249200.0,36,24,Empleado,2000000,500000,140042,47.613894,807.0,2,2,0.0,1603.0,1603.0,0.0,1,0,0,998859.0,Creciente,0
10761,9,2025-03-02 11:53:41,1283307.6,10,26,Empleado,1500000,600000,108958,42.888527,755.0,5,3,0.0,8488.0,8488.0,0.0,2,0,3,NaN,NaN,0
10762,4,2024-12-08 12:46:03,3915000.0,12,24,Independiente,4000000,2000000,306174,59.324480,716.0,9,7,0.0,5046.0,5046.0,0.0,2,1,1,NaN,NaN,0


In [17]:
# Información general
print("\nInformación del dataset:")
df.info()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10763 entries, 0 to 10762
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   tipo_credito                   10763 non-null  int64         
 1   fecha_prestamo                 10763 non-null  datetime64[ns]
 2   capital_prestado               10763 non-null  float64       
 3   plazo_meses                    10763 non-null  int64         
 4   edad_cliente                   10763 non-null  int64         
 5   tipo_laboral                   10763 non-null  object        
 6   salario_cliente                10763 non-null  int64         
 7   total_otros_prestamos          10763 non-null  int64         
 8   cuota_pactada                  10763 non-null  int64         
 9   puntaje                        10763 non-null  float64       
 10  puntaje_datacredito            10757 non-null  float64  

### 6.3 Análisis Inicial de Valores Nulos

In [18]:
# Resumen de valores nulos
null_counts = df.isnull().sum()
null_pct = (null_counts / len(df) * 100).round(2)

null_summary = pd.DataFrame({
    'Nulos': null_counts,
    'Porcentaje': null_pct
}).sort_values('Nulos', ascending=False)

print("\n" + "=" * 80)
print("RESUMEN DE VALORES NULOS")
print("=" * 80)
print(null_summary[null_summary['Nulos'] > 0])
print("\n" + "=" * 80)

total_nulls = df.isnull().sum().sum()
print(f"\nTotal de valores nulos en el dataset: {total_nulls:,}")


RESUMEN DE VALORES NULOS
                               Nulos  Porcentaje
tendencia_ingresos              2932       27.24
promedio_ingresos_datacredito   2930       27.22
saldo_mora_codeudor              590        5.48
saldo_principal                  405        3.76
saldo_mora                       156        1.45
saldo_total                      156        1.45
puntaje_datacredito                6        0.06


Total de valores nulos en el dataset: 7,175


## 7. Limpieza Inicial de Datos

### 7.1 Convertir Tipos de Datos

In [ ]:
# Crear copia para trabajar
df_clean = df.copy()

# Convertir fecha_prestamo a datetime (si existe)
if 'fecha_prestamo' in df_clean.columns:
    df_clean['fecha_prestamo'] = pd.to_datetime(df_clean['fecha_prestamo'], errors='coerce')
    print("✓ fecha_prestamo convertida a datetime")
    print(f"  Rango de fechas: {df_clean['fecha_prestamo'].min()} a {df_clean['fecha_prestamo'].max()}")
else:
    print(" Columna 'fecha_prestamo' no encontrada")

✓ fecha_prestamo convertida a datetime
  Rango de fechas: 2024-11-26 09:17:04 a 2026-04-26 18:43:52


### 7.2 Limpieza de Variable `tendencia_ingresos`

**PROBLEMA IDENTIFICADO:** La variable `tendencia_ingresos` puede contener valores numéricos que deberían ser categorías válidas.

In [ ]:
# Verificar si existe la columna
if 'tendencia_ingresos' in df_clean.columns:
    # Analizar valores actuales
    print("Valores únicos en tendencia_ingresos ANTES de limpieza:")
    print(df_clean['tendencia_ingresos'].value_counts(dropna=False).head(15))
    print(f"\nTotal de valores únicos: {df_clean['tendencia_ingresos'].nunique()}")
else:
    print(" Columna 'tendencia_ingresos' no encontrada en el dataset")

Valores únicos en tendencia_ingresos ANTES de limpieza:
tendencia_ingresos
Creciente      5294
NaN            2932
Decreciente    1291
Estable        1188
0                 7
8315              6
1000000           4
9147              2
158042            1
168750            1
3978              1
-28589            1
-566272           1
24702             1
31837             1
Name: count, dtype: int64

Total de valores únicos: 46


In [21]:
# Limpiar tendencia_ingresos si existe
if 'tendencia_ingresos' in df_clean.columns:
    # Definir valores válidos
    VALID_TENDENCIAS = ['Creciente', 'Decreciente', 'Estable']
    
    # Identificar valores inválidos
    mask_invalid = ~df_clean['tendencia_ingresos'].isin(VALID_TENDENCIAS) & df_clean['tendencia_ingresos'].notna()
    invalid_count = mask_invalid.sum()
    
    print("\n" + "=" * 80)
    print("LIMPIEZA DE tendencia_ingresos")
    print("=" * 80)
    print(f"\nValores inválidos encontrados: {invalid_count}")
    
    if invalid_count > 0:
        invalid_values = df_clean.loc[mask_invalid, 'tendencia_ingresos'].value_counts()
        print("\nValores inválidos (primeros 10):")
        print(invalid_values.head(10))
        
        # Reemplazar valores inválidos con NaN
        df_clean.loc[mask_invalid, 'tendencia_ingresos'] = np.nan
        print("\n✓ Valores inválidos reemplazados con NaN")
    else:
        print("\n✓ No se encontraron valores inválidos")
    
    # Verificar resultado
    print("\nValores únicos DESPUÉS de limpieza:")
    print(df_clean['tendencia_ingresos'].value_counts(dropna=False))
    print("\n" + "=" * 80)


LIMPIEZA DE tendencia_ingresos

Valores inválidos encontrados: 58

Valores inválidos (primeros 10):
tendencia_ingresos
0          7
8315       6
1000000    4
9147       2
158042     1
168750     1
3978       1
-28589     1
-566272    1
24702      1
Name: count, dtype: int64

✓ Valores inválidos reemplazados con NaN

Valores únicos DESPUÉS de limpieza:
tendencia_ingresos
Creciente      5294
NaN            2990
Decreciente    1291
Estable        1188
Name: count, dtype: int64



### 7.3 Verificación de Duplicados

In [ ]:
# Buscar registros duplicados
duplicates = df_clean.duplicated().sum()

print("\n" + "=" * 80)
print("VERIFICACIÓN DE DUPLICADOS")
print("=" * 80)
print(f"\nRegistros duplicados (completos): {duplicates}")

if duplicates > 0:
    print("\n Se encontraron registros duplicados")
    print("\nPrimeros 5 registros duplicados:")
    display(df_clean[df_clean.duplicated(keep=False)].head())
    
    # Opción: eliminar duplicados (comentado por defecto)
    # df_clean = df_clean.drop_duplicates()
    # print(f"✓ Duplicados eliminados. Nuevas dimensiones: {df_clean.shape}")
else:
    print("✓ No hay registros duplicados")

print("\n" + "=" * 80)


VERIFICACIÓN DE DUPLICADOS

Registros duplicados (completos): 0
✓ No hay registros duplicados



## 8. Diccionario de Datos

In [23]:
# Diccionario de datos
data_dictionary = {
    'tipo_credito': 'Tipo de producto crediticio',
    'fecha_prestamo': 'Fecha de desembolso del préstamo',
    'capital_prestado': 'Monto del préstamo en pesos',
    'plazo_meses': 'Plazo del crédito en meses',
    'edad_cliente': 'Edad del titular del crédito',
    'tipo_laboral': 'Tipo de empleo (Empleado/Independiente)',
    'salario_cliente': 'Ingreso mensual declarado',
    'total_otros_prestamos': 'Suma de otros préstamos del cliente',
    'cuota_pactada': 'Cuota mensual acordada',
    'puntaje': 'Score interno de la entidad',
    'puntaje_datacredito': 'Score de Datacrédito',
    'cant_creditosvigentes': 'Número de créditos activos',
    'huella_consulta': 'Número de consultas al historial crediticio',
    'saldo_mora': 'Saldo en mora del cliente',
    'saldo_total': 'Saldo total del crédito',
    'saldo_principal': 'Saldo del capital pendiente',
    'saldo_mora_codeudor': 'Mora del codeudor',
    'creditos_sectorFinanciero': 'Créditos en sector financiero',
    'creditos_sectorCooperativo': 'Créditos en sector cooperativo',
    'creditos_sectorReal': 'Créditos en sector real',
    'promedio_ingresos_datacredito': 'Promedio de ingresos reportados en Datacrédito',
    'tendencia_ingresos': 'Tendencia de los ingresos (Creciente/Decreciente/Estable)',
    'Pago_atiempo': 'Variable objetivo: 1=Pagó a tiempo, 0=No pagó a tiempo'
}

# Crear DataFrame del diccionario solo con columnas existentes
dict_data = []
for col in df_clean.columns:
    dict_data.append({
        'Variable': col,
        'Descripción': data_dictionary.get(col, 'Sin descripción'),
        'Tipo': str(df_clean[col].dtype)
    })

dict_df = pd.DataFrame(dict_data)

print("\n" + "=" * 80)
print("DICCIONARIO DE DATOS")
print("=" * 80)
display(dict_df)
print("=" * 80)


DICCIONARIO DE DATOS


,Variable,Descripción,Tipo
0,tipo_credito,Tipo de producto crediticio,int64
1,fecha_prestamo,Fecha de desembolso del préstamo,datetime64[ns]
2,capital_prestado,Monto del préstamo en pesos,float64
3,plazo_meses,Plazo del crédito en meses,int64
4,edad_cliente,Edad del titular del crédito,int64
5,tipo_laboral,Tipo de empleo (Empleado/Independiente),object
6,salario_cliente,Ingreso mensual declarado,int64
7,total_otros_prestamos,Suma de otros préstamos del cliente,int64
8,cuota_pactada,Cuota mensual acordada,int64
9,puntaje,Score interno de la entidad,float64


## 9. Guardar Datos Limpios

Guardamos los datos en la carpeta `data/interim/` para usar en el EDA.

In [24]:
# Definir rutas de salida
OUTPUT_CSV = DATA_INTERIM / 'creditos_limpio.csv'
OUTPUT_PKL = DATA_INTERIM / 'creditos_limpio.pkl'

# Guardar en CSV
df_clean.to_csv(OUTPUT_CSV, index=False)
print(f"✓ Datos guardados en CSV: {OUTPUT_CSV}")
print(f"  Dimensiones: {df_clean.shape[0]:,} filas × {df_clean.shape[1]} columnas")

# También guardar en pickle (más eficiente para Python)
df_clean.to_pickle(OUTPUT_PKL)
print(f"✓ Datos guardados en Pickle: {OUTPUT_PKL}")

print(f"\nArchivos generados:")
print(f"  1. {OUTPUT_CSV.relative_to(PROJECT_ROOT)}")
print(f"  2. {OUTPUT_PKL.relative_to(PROJECT_ROOT)}")

✓ Datos guardados en CSV: d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\interim\creditos_limpio.csv
  Dimensiones: 10,763 filas × 23 columnas
✓ Datos guardados en Pickle: d:\Work\Cursos\Data Science\M5_Fundamentos_de_Nube_y_Ciencia_de_Datos_de_Producción\PI\PI_M5\mlops_pipeline\src\data\interim\creditos_limpio.pkl

Archivos generados:
  1. data\interim\creditos_limpio.csv
  2. data\interim\creditos_limpio.pkl


## 10. Resumen Final

In [25]:
# Calcular número de valores inválidos limpiados
invalid_tendencia = mask_invalid.sum() if 'tendencia_ingresos' in df_clean.columns else 0

print("\n" + "=" * 80)
print("RESUMEN DE CARGA DE DATOS")
print("=" * 80)
print(f"\n✓ Archivo cargado: {FILE_PATH.name}")
print(f"✓ Registros: {len(df_clean):,}")
print(f"✓ Variables: {len(df_clean.columns)}")

if 'fecha_prestamo' in df_clean.columns:
    fecha_min = df_clean['fecha_prestamo'].min()
    fecha_max = df_clean['fecha_prestamo'].max()
    if pd.notna(fecha_min) and pd.notna(fecha_max):
        print(f"✓ Período: {fecha_min.date()} a {fecha_max.date()}")

print(f"\nLimpieza aplicada:")
if invalid_tendencia > 0:
    print(f"  • tendencia_ingresos: {invalid_tendencia} valores inválidos convertidos a NaN")
if 'fecha_prestamo' in df_clean.columns:
    print(f"  • fecha_prestamo: Convertida a datetime")
print(f"  • Duplicados: {duplicates} encontrados")

print(f"\nArchivos generados:")
print(f"  • {OUTPUT_CSV.name}")
print(f"  • {OUTPUT_PKL.name}")

print(f"\n✓ Dataset listo para EDA")
print("\n" + "=" * 80)


RESUMEN DE CARGA DE DATOS

✓ Archivo cargado: Base_de_datos.xlsx
✓ Registros: 10,763
✓ Variables: 23
✓ Período: 2024-11-26 a 2026-04-26

Limpieza aplicada:
  • tendencia_ingresos: 58 valores inválidos convertidos a NaN
  • fecha_prestamo: Convertida a datetime
  • Duplicados: 0 encontrados

Archivos generados:
  • creditos_limpio.csv
  • creditos_limpio.pkl

✓ Dataset listo para EDA

